In [3]:
import pandas as pd
import requests as rq
import json
import urllib.parse as urlparse
from urllib.parse import parse_qs
from urllib.parse import unquote
from datetime import datetime

Reference page
https://docs.microsoft.com/en-us/rest/api/cost-management/retail-prices/azure-retail-prices

In [20]:
# Helper method for extracting the number of the next skip value
def extract_skip(url):
    decoded = unquote(url)
    parsed = urlparse.urlparse(decoded)
    result = parse_qs(parsed.query)['$skip'][0]
    return int(result)

def read_page(items, url):
    reponse = rq.get(url)
    page = json.loads(reponse.text)
    items += page['Items'] 
    next_page = page['NextPageLink']
    return items, next_page

In [21]:
def read_and_extract(items, next_page):
    while next_page:
        items, next_page = read_page(items, next_page)    
        if not next_page:
            break
    
        # only print for every 1000 pages
        skip = extract_skip(next_page)
        if skip % 1000 == 0 :
            now = datetime.now()
            dt_string = now.strftime('%H:%M:%S')
            print (dt_string, ': ', next_page)
    
    print('===== Done ======')
    return items

### Compute

In [ ]:
compute_url = "https://prices.azure.com/api/retail/prices?currencyCode='USD'&$filter=serviceFamily%20eq%20'Compute'"
items = []   
items, next_page = read_page(items, compute_url)
items = read_and_extract(items, next_page)

In [6]:
# pickle
azure_data = pd.DataFrame(items)
azure_data.to_pickle('./data/azure.compute.xz.pkl', compression='xz')

In [7]:
azure_src = './data/azure.compute.xz.pkl' 
azure_data = pd.read_pickle(azure_src, compression="xz")

In [9]:
azure_data

,currencyCode,tierMinimumUnits,retailPrice,unitPrice,armRegionName,location,effectiveStartDate,meterId,meterName,productId,...,skuName,serviceName,serviceId,serviceFamily,unitOfMeasure,type,isPrimaryMeterRegion,armSkuName,effectiveEndDate,reservationTerm
0,USD,0.0,1.025809,1.025809,southindia,IN South,2021-06-01T00:00:00Z,000009d0-057f-5f2b-b7e9-9e26add324a8,D14/DS14 Spot,DZH318Z0BPVW,...,D14 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_D14,NaN,NaN
1,USD,0.0,0.410550,0.410550,southindia,IN South,2021-06-01T00:00:00Z,000009d0-057f-5f2b-b7e9-9e26add324a8,D14/DS14 Spot,DZH318Z0BPVW,...,D14 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,DevTestConsumption,True,Standard_D14,NaN,NaN
2,USD,0.0,0.422400,0.422400,eastasia,AP East,2021-04-15T00:00:00Z,00014e7d-fff9-54dd-962e-4e992887ad3c,D64s v5 Spot,DZH318Z08M9T,...,Standard_D64s_v5 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,DevTestConsumption,True,Standard_D64s_v5,NaN,NaN
3,USD,0.0,1.433600,1.433600,eastasia,AP East,2021-04-15T00:00:00Z,00014e7d-fff9-54dd-962e-4e992887ad3c,D64s v5 Spot,DZH318Z08M9T,...,Standard_D64s_v5 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_D64s_v5,NaN,NaN
4,USD,0.0,0.062988,0.062988,uknorth,UK North,2019-10-16T00:00:00Z,0001e46a-9285-5fa8-b48a-240e307a24f7,A3 Spot,DZH318Z0BPVJ,...,A3 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,DevTestConsumption,True,Standard_A3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187560,USD,0.0,0.475000,0.475000,southafricawest,ZA West,2021-04-15T00:00:00Z,fffdd929-47ed-5006-98fe-4676b210cd66,D96d v5 Low Priority,DZH318Z08M9F,...,Standard_D96d_v5 Low Priority,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,DevTestConsumption,True,Standard_D96d_v5,NaN,NaN
187561,USD,0.0,0.178000,0.178000,koreasouth,KR South,2021-04-15T00:00:00Z,fffe202e-74df-5cb1-aa4e-0a3f87b290f6,D8ds v5 Spot,DZH318Z08MC5,...,Standard_D8ds_v5 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_D8ds_v5,NaN,NaN
187562,USD,0.0,0.052400,0.052400,koreasouth,KR South,2021-04-15T00:00:00Z,fffe202e-74df-5cb1-aa4e-0a3f87b290f6,D8ds v5 Spot,DZH318Z08MC5,...,Standard_D8ds_v5 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,DevTestConsumption,True,Standard_D8ds_v5,NaN,NaN
187563,USD,0.0,0.131000,0.131000,brazilsoutheast,BR Southeast,2021-05-01T00:00:00Z,ffff626b-776e-58b7-9c95-842102f6309e,E2s v5,DZH318Z096T1,...,E2s v5,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,DevTestConsumption,True,Standard_E2s_v5,NaN,NaN


### Storage

In [26]:
storage_url = "https://prices.azure.com/api/retail/prices?$filter=serviceName%20eq%20%27Storage%27"
items = []   
items, next_page = read_page(items, storage_url)
items = read_and_extract(items, next_page)

17:13:34 :  https://prices.azure.com:443/api/retail/prices?$filter=serviceName%20eq%20%27Storage%27&$skip=1000
17:13:41 :  https://prices.azure.com:443/api/retail/prices?$filter=serviceName%20eq%20%27Storage%27&$skip=2000
17:13:49 :  https://prices.azure.com:443/api/retail/prices?$filter=serviceName%20eq%20%27Storage%27&$skip=3000
17:13:56 :  https://prices.azure.com:443/api/retail/prices?$filter=serviceName%20eq%20%27Storage%27&$skip=4000
17:14:04 :  https://prices.azure.com:443/api/retail/prices?$filter=serviceName%20eq%20%27Storage%27&$skip=5000
17:14:10 :  https://prices.azure.com:443/api/retail/prices?$filter=serviceName%20eq%20%27Storage%27&$skip=6000
17:14:19 :  https://prices.azure.com:443/api/retail/prices?$filter=serviceName%20eq%20%27Storage%27&$skip=7000
17:14:27 :  https://prices.azure.com:443/api/retail/prices?$filter=serviceName%20eq%20%27Storage%27&$skip=8000
17:14:35 :  https://prices.azure.com:443/api/retail/prices?$filter=serviceName%20eq%20%27Storage%27&$skip=9000
1

In [29]:
# pickle
storage_data = pd.DataFrame(items)
storage_data.to_pickle('./data/azure.storage.xz.pkl', compression='xz')

In [30]:
storage_data.head(10)

,currencyCode,tierMinimumUnits,retailPrice,unitPrice,armRegionName,location,effectiveStartDate,meterId,meterName,productId,...,productName,skuName,serviceName,serviceId,serviceFamily,unitOfMeasure,type,isPrimaryMeterRegion,armSkuName,reservationTerm
0,USD,0.0,10.56000,10.56000,uksouth,UK South,2018-11-01T00:00:00Z,0001d427-82df-4d83-8ab2-b60768527e08,E10 Disks,DZH318Z0BP88,...,Standard SSD Managed Disks,E10 LRS,Storage,DZH317F1HKN0,Storage,1/Month,Consumption,True,,NaN
1,USD,0.0,0.06190,0.06190,norwayeast,NO East,2019-12-01T00:00:00Z,00027356-c92d-4d1c-bf45-4503a328cf3e,ZRS Data Stored,DZH318Z0BNWT,...,Standard Page Blob v2,Standard ZRS,Storage,DZH317F1HKN0,Storage,1 GB/Month,Consumption,True,,NaN
2,USD,0.0,0.10634,0.10634,brazilsoutheast,BR Southeast,2020-08-25T00:00:00Z,000e4dbc-9b46-5a2b-a833-57d93fdc820f,LRS Data Stored,DZH318Z0BP09,...,Files v2,Standard LRS,Storage,DZH317F1HKN0,Storage,1 GB/Month,Consumption,False,,NaN
3,USD,0.0,0.10634,0.10634,brazilsoutheast,BR Southeast,2020-08-25T00:00:00Z,000e4dbc-9b46-5a2b-a833-57d93fdc820f,LRS Data Stored,DZH318Z0BP0B,...,Files,Standard LRS,Storage,DZH317F1HKN0,Storage,1 GB/Month,Consumption,True,,NaN
4,USD,512000.0,0.04870,0.04870,brazilsoutheast,BR Southeast,2020-08-25T00:00:00Z,00107f02-493a-5afe-9da4-a44ba7acabf3,Hot ZRS Data Stored,DZH318Z0BPH7,...,General Block Blob v2,Hot ZRS,Storage,DZH317F1HKN0,Storage,1 GB/Month,Consumption,True,,NaN
5,USD,0.0,0.05298,0.05298,brazilsoutheast,BR Southeast,2020-08-25T00:00:00Z,00107f02-493a-5afe-9da4-a44ba7acabf3,Hot ZRS Data Stored,DZH318Z0BPH7,...,General Block Blob v2,Hot ZRS,Storage,DZH317F1HKN0,Storage,1 GB/Month,Consumption,True,,NaN
6,USD,51200.0,0.05090,0.05090,brazilsoutheast,BR Southeast,2020-08-25T00:00:00Z,00107f02-493a-5afe-9da4-a44ba7acabf3,Hot ZRS Data Stored,DZH318Z0BPH7,...,General Block Blob v2,Hot ZRS,Storage,DZH317F1HKN0,Storage,1 GB/Month,Consumption,True,,NaN
7,USD,0.0,0.00500,0.00500,centralindia,IN Central,2020-03-01T00:00:00Z,00109a87-d6f6-4098-92cc-899900f65d53,Archive Other Operations,DZH318Z0C120,...,General Block Blob v2 Hierarchical Namespace,Archive ZRS,Storage,DZH317F1HKN0,Storage,10K,Consumption,False,,NaN
8,USD,0.0,0.00500,0.00500,centralindia,IN Central,2020-03-01T00:00:00Z,00109a87-d6f6-4098-92cc-899900f65d53,Archive Other Operations,DZH318Z0C120,...,General Block Blob v2 Hierarchical Namespace,Archive RA-GRS,Storage,DZH317F1HKN0,Storage,10K,Consumption,False,,NaN
9,USD,0.0,0.00500,0.00500,centralindia,IN Central,2020-03-01T00:00:00Z,00109a87-d6f6-4098-92cc-899900f65d53,Archive Other Operations,DZH318Z0C120,...,General Block Blob v2 Hierarchical Namespace,Archive LRS,Storage,DZH317F1HKN0,Storage,10K,Consumption,True,,NaN
